# Library

In [ ]:
!pip install transformers
!pip install Sastrawi
!pip install langdetect

In [ ]:
# import library untuk memuat dan menganalisis data
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

# import library untuk text cleaning
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import re

# import library untuk deteksi bahasa
from langdetect import detect, LangDetectException

# import pipeline
from sklearn.pipeline import Pipeline

# import library untuk membuat model
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline
from tqdm import tqdm
import torch
import os

# import library untuk menyimpan model yang sudah dibuat
import joblib

# Memuat Data

In [ ]:
Data = "https://github.com/keaganwjy/datathon/raw/refs/heads/main/reviews_genshin_impact_raw.csv"
jumlah_baris = 2000 # jumlah baris yang akan dimasukan.as_integer_ratio
df_raw = pd.read_csv(Data) # membaca CSV
df = df_raw.head(jumlah_baris).copy() # menggunakan .head() untuk mengambil baris teratas secara langsung
df.head()

,Nama Pengguna,Teks Ulasan,Rating Bintang,Tanggal Ulasan,Jumlah Likes
0,Pengguna Google,pelit pengen karakter aja susah malah dapet yg...,1,2025-06-20 14:40:33,0
1,Pengguna Google,farming artefak engga pernah dikasih stat/sub ...,1,2025-06-20 14:00:33,0
2,Pengguna Google,"udah ngga worth it, perbaiki diri mu genshin, ...",1,2025-06-20 13:49:19,0
3,Pengguna Google,kasih fitur skip.,1,2025-06-20 13:22:21,0
4,Pengguna Google,peak update,5,2025-06-20 13:16:04,0


# Data Cleaning Dengan Pipeline

1. Penyesuaian Data
2. Pembersihan Text
3. Deteksi Bahasa

In [ ]:
# class untuk menyesuaikan tipe data
class PenyesuaianTipeData(BaseEstimator,TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # mengubah 'Tanggal Ulasan' menjadi format datetime
        # ini memungkinkan kita untuk melakukan analisis berbasis waktu nanti
        X['Tanggal Ulasan'] = pd.to_datetime(X['Tanggal Ulasan'])

        # memastikan 'Rating Bintang' dan 'Jumlah Likes' adalah angka (integer)
        # error='coerce' akan mengubah nilai yang tidak bisa diubah menjadi angka (misal: teks) menjadi NaN (kosong)
        X['Rating Bintang'] = pd.to_numeric(X['Rating Bintang'], errors='coerce')
        X['Jumlah Likes'] = pd.to_numeric(X['Jumlah Likes'], errors='coerce')

        # kita bisa membuang baris yang rating atau likes-nya menjadi NaN, atau mengisinya dengan 0
        X.dropna(subset=['Rating Bintang'], inplace=True) # rating wajib ada
        # kode baru (praktik terbaik dan aman)
        X['Jumlah Likes'] = X['Jumlah Likes'].fillna(0)

        # mengubah tipe data menjadi integer untuk efisiensi memori
        X['Rating Bintang'] = X['Rating Bintang'].astype(int)
        X['Jumlah Likes'] = X['Jumlah Likes'].astype(int)

        return X

In [ ]:
# class untuk pembersihan teks
class PembersihanText(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # inisialisasi stemmer dan stopword remover
        factory_stemmer = StemmerFactory()
        stemmer = factory_stemmer.create_stemmer()
        factory_stopword = StopWordRemoverFactory()
        stopword_remover = factory_stopword.create_stop_word_remover()

        # buat kamus slang yang lebih lengkap (ini bisa terus Anda kembangkan)
        slang_dict = {
        'bgt': 'banget', 'gak': 'tidak', 'ga': 'tidak', 'kalo': 'kalau', 'gacha': 'gacha',
        'dgn': 'dengan', 'krn': 'karena', 'yg': 'yang', 'utk': 'untuk', 'mantap': 'mantap',
        'keren': 'keren', 'bug': 'bug', 'ngebug': 'bug', 'loding': 'loading', 'ngelek': 'lag',
        'ngeframe': 'frame', 'drop': 'drop', 'jelek': 'jelek', 'bangettt': 'banget',
        'sih': '', 'nya': '', 'aja': 'saja', 'kok': '', 'sih': ''
        }

        def preprocess_text(text):
            # pastikan input adalah string
            if not isinstance(text, str):
                return ""
            # 1. case folding
            text = text.lower()
            # 2. hapus noise (URL, mention, hashtag, karakter non-alfabet)
            text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
            text = re.sub(r'\@\w+|\#', '', text)
            text = re.sub(r'[^a-z\s]', '', text)
            # 3. normalisasi kata slang
            words = text.split()
            normalized_words = [slang_dict[word] if word in slang_dict else word for word in words]
            text = " ".join(normalized_words)
            # 4. Stopword Removal
            text = stopword_remover.remove(text)
            # 5. stemming
            text = stemmer.stem(text)
            return text

        # menggunakan .copy() untuk menghindari SettingWithCopyWarning
        X_processed = X.copy()
        X_processed['Ulasan Bersih'] = X_processed['Teks Ulasan'].apply(preprocess_text)
        return X_processed





In [ ]:
# class untuk  medeteksi bahasa
class DeteksiBahasa(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        kolom_krusial = ['Ulasan Bersih', 'Rating Bintang', 'Tanggal Ulasan']
        X.dropna(subset=kolom_krusial, inplace=True)

        # cek jumlah baris setelah pembersihan untuk melihat perbedaannya
        X.reset_index(drop=True, inplace=True)

        # 1. Definisikan fungsi yang aman (pastikan nama ini yanga digunakan)
        def deteksi_bahasa_aman(teks):
            # cek apakah inputnya bukan string (misalnya NaN)
            if pd.isna(teks) or not isinstance(teks, str):
                return 'undefined'

            # cek apakah teks terlalu pendek
            if len(teks.strip()) < 10:
                return 'pendek'

            # coba deteksi bahasa
            try:
                return detect(teks)
            except LangDetectException:
                return 'error'

    # 2. Terapkan fungsi yang benar pada dataframe anda
    # asumsikan 'X' adalah dataframe anda
    # perbaikan di sini: Panggil 'deteksi_bahasa_aman'
        X['bahasa'] = X['Ulasan Bersih'].apply(deteksi_bahasa_aman)

        # hasil
        print("\nDistribusi Bahasa pada Dataset")
        print(X['bahasa'].value_counts())
        return X



# Labelling

In [ ]:
class Labelling(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self,X):
        os.environ["HF_HUB_DOWNLOAD_TIMEOUT"] = "60"  # Timeout jadi 60 detik
        classifier = pipeline(
            "zero-shot-classification",
            model="cahya/distilbert-base-indonesian",
            device=0  # atau -1 jika tidak ada GPU
        )

        print("Model berhasil dimuat.")

        # --- Definisikan Fungsi Pelabelan ---
        # Daftar kategori final Anda
        candidate_labels = ['Cerita', 'Gameplay', 'Grafis', 'Bugs & Error', 'Optimalisasi', 'Monetisasi & Gacha', 'Komunitas'] # Tambah/ubah sesuai kebutuhan

        def get_auto_labels(text):
            # Ambang batas skor agar sebuah label dianggap relevan
            threshold = 0.60 # Anda bisa menyesuaikan ini nanti

            try:
                result = classifier(text, candidate_labels, multi_label=True)

                # Ambil label yang skornya di atas ambang batas
                labels = [label for label, score in zip(result['labels'], result['scores']) if score > threshold]

                # Jika tidak ada label di atas ambang batas, kembalikan list kosong
                return labels if labels else []
            except Exception as e:
                # Jika terjadi error pada teks tertentu
                print(f"Error pada teks: {text[:50]}... | Error: {e}")
                return ["error"]

        # --- Terapkan pada Sampel Kecil untuk Uji Coba ---
        # PENTING: Jalankan pada sampel kecil dulu (misal: 1000-2000 baris)
        # untuk menghemat waktu dan memvalidasi proses.
        X_sample = X.head(2000).copy()
        X_sample = X_sample[X_sample['Ulasan Bersih'].notna() & (X_sample['Ulasan Bersih'].str.strip() != "")]

        X_sample['auto_labels'] = [
            classifier(text, candidate_labels, multi_label=True)
            for text in tqdm(X_sample['Ulasan Bersih'])
        ]
        print("Pelabelan otomatis selesai.")

        # Mari kita lihat hasilnya. Hasilnya akan sedikit berbeda, berupa dictionary.
        # Kita akan ekstrak labelnya saja.
        def extract_labels_from_result(result, threshold=0.60):
            return [label for label, score in zip(result['labels'], result['scores']) if score > threshold]

        X_sample['auto_labels_list'] = [extract_labels_from_result(res) for res in X_sample['auto_labels']]


        # Tampilkan kolom-kolom yang relevan
        print("\nContoh Hasil Pelabelan Otomatis:")
        print(X_sample[['Ulasan Bersih', 'auto_labels_list']].head(10))

        return X_sample

# Execute

In [ ]:
Pipe = Pipeline([
    ("Penyesuaian",PenyesuaianTipeData()),
    ("Pembersihan",PembersihanText()),
    ("Deteksi",DeteksiBahasa()),
    ("Labelling",Labelling())
])

In [ ]:
model_genshin = Pipe.transform(df)


Distribusi Bahasa pada Dataset
bahasa
id        1177
pendek     308
tl         153
en         103
lt          43
no          24
so          21
da          18
sl          15
sw          14
nl          13
et          13
sq          12
tr          10
ca          10
ro          10
fi           9
af           9
hr           9
it           7
cy           7
fr           3
pt           2
sk           2
de           2
lv           2
es           2
sv           1
hu           1
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Model berhasil dimuat.


100%|██████████| 1979/1979 [05:05<00:00,  6.49it/s]

Pelabelan otomatis selesai.

Contoh Hasil Pelabelan Otomatis:
                                       Ulasan Bersih  \
0          pelit ken karakter susah malah dapet lain   
1  farming artefak engga pernah kasih statsub sta...   
2  udah ngga worth it baik diri mu genshin dengar...   
3                                   kasih fitur skip   
4                                        peak update   
5                                               seru   
6  game ada kembang sama sekali scene kaku desain...   
7  kuota ku habis cuman mendondlod data beri pili...   
8                               bagus banget gamenya   
9      game e apik poll lek hp ne kentang ojok maksa   

                                    auto_labels_list  
0  [Monetisasi & Gacha, Komunitas, Gameplay, Ceri...  
1  [Monetisasi & Gacha, Gameplay, Komunitas, Ceri...  
2  [Monetisasi & Gacha, Cerita, Gameplay, Komunit...  
3                     [Monetisasi & Gacha, Gameplay]  
4                               [Monetisasi & 

In [ ]:
# model_genshin.to_csv("genshin_impact_clean.csv", index=False)

# Mencoba Pipe di Data lain

In [ ]:
Data_wuwa = "https://github.com/keaganwjy/datathon/raw/refs/heads/main/reviews_wuthering_waves_raw.csv"
jumlah_baris = 2000 # jumlah baris yang akan dimasukan.as_integer_ratio
df_raw = pd.read_csv(Data_wuwa) # membaca CSV
df_wuwa = df_raw.head(jumlah_baris).copy() # menggunakan .head() untuk mengambil baris teratas secara langsung
df_wuwa.head()

,Nama Pengguna,Teks Ulasan,Rating Bintang,Tanggal Ulasan,Jumlah Likes
0,Pengguna Google,game yang menarik,5,2025-06-20 14:22:34,0
1,Pengguna Google,"grafik, karakter dan optimisasi sangat sangat ...",5,2025-06-20 13:50:34,0
2,Pengguna Google,saya berharap setelah story mendapatkan lebih ...,4,2025-06-20 12:50:21,0
3,Pengguna Google,Samsung A55 main di settingan low sambil pakai...,3,2025-06-20 12:35:52,0
4,Pengguna Google,weel played aku kasih bintang satu karena aku ...,1,2025-06-20 12:20:46,6


In [ ]:
model_wuwa = Pipe.transform(df_wuwa)


Distribusi Bahasa pada Dataset
bahasa
id        993
pendek    395
en        299
tl        117
so         25
da         19
et         17
ro         16
no         16
lt         15
it         11
sw         10
fi          8
hr          8
af          8
nl          8
cy          5
fr          5
sl          4
lv          4
ca          3
es          2
tr          2
sq          2
pl          2
vi          1
pt          1
cs          1
de          1
hu          1
sv          1
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Model berhasil dimuat.


100%|██████████| 1972/1972 [04:36<00:00,  7.13it/s]

Pelabelan otomatis selesai.

Contoh Hasil Pelabelan Otomatis:
                                       Ulasan Bersih  \
0                                         game tarik   
1  grafik karakter optimisasi sangat sangat bagus...   
2             harap story dapat lebih banyak diamond   
3  samsung a main settingan low pakai cooler baru...   
4  weel played aku kasih bintang satu aku nguli b...   
5                              terimakasih game baik   
6                  seru banget terus dev baik banget   
7                             sangat sesuai expetasi   
8                                      it is so good   
9  gamenya bagus sekali pake banget banget donk d...   

                                    auto_labels_list  
0                                                 []  
1  [Grafis, Optimalisasi, Cerita, Bugs & Error, K...  
2  [Optimalisasi, Grafis, Cerita, Komunitas, Bugs...  
3                                                 []  
4                                             

In [ ]:
model_wuwa.to_csv('wuthering_waves_clean.csv')

In [ ]:
Data_hsr = "https://github.com/keaganwjy/datathon/raw/refs/heads/main/reviews_honkai_star_rail_raw.csv"
jumlah_baris = 2000 # jumlah baris yang akan dimasukan.as_integer_ratio
df_raw = pd.read_csv(Data_hsr) # membaca CSV
df_hsr = df_raw.head(jumlah_baris).copy() # menggunakan .head() untuk mengambil baris teratas secara langsung
df_hsr.head()

,Nama Pengguna,Teks Ulasan,Rating Bintang,Tanggal Ulasan,Jumlah Likes
0,Pengguna Google,"The story is just peak, especially penacony. I...",1,2025-06-20 13:08:34,0
1,Pengguna Google,"perfect graphics, in terms of gameplay is also...",5,2025-06-20 12:33:18,0
2,Pengguna Google,PEAK STORY,5,2025-06-20 08:06:53,0
3,Pengguna Google,keren,5,2025-06-20 07:49:05,0
4,Pengguna Google,overall good,4,2025-06-20 05:54:24,0


In [ ]:
model_hsr = Pipe.transform(df_hsr)



Distribusi Bahasa pada Dataset
bahasa
id        1216
en         229
pendek     224
tl         131
so          24
no          18
lt          17
pl          13
it          12
ro          12
et          11
ca           9
hr           9
sw           9
cy           8
af           8
da           6
fi           6
sl           6
sq           6
nl           5
sk           5
fr           5
pt           4
tr           2
es           1
cs           1
hu           1
de           1
sv           1
Name: count, dtype: int64


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


Model berhasil dimuat.


100%|██████████| 1968/1968 [04:07<00:00,  7.95it/s]

Pelabelan otomatis selesai.

Contoh Hasil Pelabelan Otomatis:
                                       Ulasan Bersih auto_labels_list
0  the story is just peak especially penacony if ...               []
1  perfect graphics in terms of gameplay is also ...               []
2                                         peak story               []
3                                              keren               []
4                                       overall good               []
5        kurang bintang aku tambahin game bagus juga               []
6                           di perhati story telling               []
7                            login error terus tolol               []
8                    langsung bintang wak no comment               []
9  bagus pelit n banyak animasi alur cerita bagus...               []


In [ ]:
model_hsr.to_csv('honkai_star_rail_clean.csv', index=False)